In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
#General Imports
%cd '/content/gdrive/My Drive/BE Project/CLSTM /Files/'
import datasets
import get_discharge_summaries
import concat_and_split
from constants import MIMIC_3_DIR, DATA_DIR

import numpy as np
import pandas as pd

from collections import Counter, defaultdict
import csv
import math
import operator

/content/gdrive/My Drive/BE Project/CLSTM /Files


In [0]:
data_path='/content/gdrive/My Drive/BE Project/CLSTM /Data/'

In [0]:
Y = 'full' #use all available labels in the dataset for prediction
notes_file = '%s/NOTEEVENTS.csv' % MIMIC_3_DIR # raw note events downloaded from MIMIC-III
vocab_size = 'full' #don't limit the vocab size to a specific number
vocab_min = 3 #discard tokens appearing in fewer than this many documents

In [0]:
dfproc = pd.read_csv('%s/PROCEDURES_ICD.csv' % MIMIC_3_DIR)
dfdiag = pd.read_csv('%s/DIAGNOSES_ICD.csv' % MIMIC_3_DIR)

In [0]:
dfdiag['absolute_code'] = dfdiag.apply(lambda row: str(datasets.reformat(str(row[4]), True)), axis=1)
dfproc['absolute_code'] = dfproc.apply(lambda row: str(datasets.reformat(str(row[4]), False)), axis=1)

In [0]:
dfdiag.head()

In [0]:
dfproc.head()

In [0]:
dfcodes = pd.concat([dfdiag, dfproc])

In [0]:
print(dfdiag.shape)
print(dfproc.shape)
print(dfcodes.shape)

(651047, 6)
(240095, 6)
(891142, 6)


In [0]:
dfcodes.to_csv('%s/ALL_CODES.csv' % MIMIC_3_DIR, index=False,
               columns=['ROW_ID', 'SUBJECT_ID', 'HADM_ID', 'SEQ_NUM', 'absolute_code'],
               header=['ROW_ID', 'SUBJECT_ID', 'HADM_ID', 'SEQ_NUM', 'ICD9_CODE'])

In [0]:
#In the full dataset (not just discharge summaries)
df_allcodes = pd.read_csv(data_path+'ALL_CODES.csv', dtype={"ICD9_CODE": str})
print(len(df_allcodes))
print(len(df_allcodes['ICD9_CODE'].unique()))

891142
8994


In [0]:
df_allcodes.head()

,ROW_ID,SUBJECT_ID,HADM_ID,SEQ_NUM,ICD9_CODE
0,1297,109,172335,1.0,403.01
1,1298,109,172335,2.0,486
2,1299,109,172335,3.0,582.81
3,1300,109,172335,4.0,585.5
4,1301,109,172335,5.0,425.4


Total number of ICD codes in diag_ics and proc_icd: **8994**

Read all rows from NOTEEVENTS

Get only those rows that have category = "Discharge summary"

Tokenize the text part and pick only four columns to form the final **disch_full** csv file 

Sub, Hadm_id, Charttime and the text

In [0]:
#This reads all notes, selects only the discharge summaries, and tokenizes them, returning the output filename
disch_full_file = get_discharge_summaries.write_discharge_summaries(out_file="%s/disch_full.csv" % MIMIC_3_DIR)

In [0]:
df_dischfull = pd.read_csv(data_path+'disch_full.csv')

In [0]:
df_dischfull.head()

In [0]:
not_null = df_dischfull[df_dischfull.CHARTTIME.notnull()]
not_null.head(10)

The number of unique admission ids in the NOTEEVENTS table is **52726**

In [0]:
len(df_dischfull['HADM_ID'].unique())

Types should mostly be the length of the vocabulary in NOTEEVENTS

Sorting the dataframe of the processed NOTEEVENTS table 

In [0]:
#Let's sort by SUBJECT_ID and HADM_ID to make a correspondence with the MIMIC-3 label file
df_dischfull = df_dischfull.sort_values(['SUBJECT_ID', 'HADM_ID'])

In [0]:
df_dischfull.shape

Out of 59652 rows in the dischfull dataframe only 52726 rows contain a unique admission id

In [0]:
df_dischfull.head()

In [0]:
dflabel = pd.read_csv(data_path+'ALL_CODES.csv')
dflabel = dflabel.sort_values(['SUBJECT_ID', 'HADM_ID'])

In [0]:
dflabel.head()

In [0]:
len(df_dischfull['HADM_ID'].unique()), len(dflabel['HADM_ID'].unique())

(52726, 58976)

Since there are more unique admission ids in the label dataframe we can infer that there exist some admission ids that do not have a corresponding discharge summary entry in the noteevents table. Hence they are of no use to use. Should probably filter them out.

In [0]:
#Let's filter out these HADM_ID's
hadm_ids = set(df_dischfull['HADM_ID'])
with open('%s/ALL_CODES.csv' % MIMIC_3_DIR, 'r') as lf:
    with open('%s/ALL_CODES_filtered.csv' % MIMIC_3_DIR, 'w') as of:
        w = csv.writer(of)
        w.writerow(['SUBJECT_ID', 'HADM_ID', 'ICD9_CODE', 'ADMITTIME', 'DISCHTIME'])
        r = csv.reader(lf)
        #header
        next(r)
        for i,row in enumerate(r):
            hadm_id = int(row[2])
            if hadm_id in hadm_ids:
                w.writerow(row[1:3] + [row[-1], '', ''])

In [0]:
df_newlabel = pd.read_csv(data_path+'ALL_CODES_filtered.csv', index_col=None)

In [0]:
len(df_newlabel['HADM_ID'].unique())

52726

In [0]:
df_newlabel = df_newlabel.sort_values(['SUBJECT_ID', 'HADM_ID']) 
df_newlabel.to_csv(data_path+'ALL_CODES_filtered.csv', index=False)

In [0]:
df_newlabel.head()

In [0]:
sorted_file = '%s/disch_full.csv' % MIMIC_3_DIR
df_dischfull.to_csv(sorted_file, index=False)

In [0]:
df_dischfull.head()

In [0]:
print(df_dischfull.shape)

In [0]:
df_newlabel.head()

In [0]:
print(df_newlabel.shape)

Input files are 
1. all codes filtered (basically all icd codes from proc and diag)
2. disch full (noteevents table sorted by sub and hadm id)

In [0]:
labeled = concat_and_split.concat_data(data_path+'ALL_CODES_filtered.csv', sorted_file)

In [0]:
test=pd.read_csv(data_path+'notes_labeled.csv')

In [0]:
test.head(10)

train_full_hadm_ids = 47723

test_full_hadm_ids = 3372

dev_full_hadm_ids = 1631

Total = 52726


In [0]:
fname = '%s/notes_labeled.csv' % MIMIC_3_DIR
base_name = "%s/disch" % MIMIC_3_DIR #for output
tr, dv, te = concat_and_split.split_data(fname, base_name=base_name)

In [0]:
tr = pd.read_csv(data_path+'disch_train_split.csv')
tr.head()

In [0]:
dv = pd.read_csv(data_path+'disch_dev_split.csv')
dv.head()

In [0]:
te = pd.read_csv(data_path+'disch_test_split.csv')
te.head()

In [0]:
build_vocab = pd.read_csv(data_path+'notes_labeled.csv')

In [0]:
import tensorflow
import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

def vectorize_notes(col, MAX_NB_WORDS, verbose = True):
    """Takes a note column and encodes it into a series of integer
        Also returns the dictionnary mapping the word to the integer"""
    tokenizer = Tokenizer(num_words = MAX_NB_WORDS)
    tokenizer.fit_on_texts(col)
    data = tokenizer.texts_to_sequences(col)
    note_length =  [len(x) for x in data]
    vocab = tokenizer.word_index
    MAX_VOCAB = len(vocab)
    if verbose:
        print('Vocabulary size: %s' % MAX_VOCAB)
        print('Average note length: %s' % np.mean(note_length))
        print('Max note length: %s' % np.max(note_length))
    return data, vocab, MAX_VOCAB

def pad_notes(data, MAX_SEQ_LENGTH):
    data = pad_sequences(data, maxlen = MAX_SEQ_LENGTH)
    return data, data.shape[1]

Using TensorFlow backend.


In [0]:
# Finding Dictionary and Vocab Size
MAX_VOCAB= None
data_vectorized, dictionary, MAX_VOCAB = vectorize_notes(build_vocab.TEXT, MAX_VOCAB, verbose = True)

Vocabulary size: 150485
Average note length: 1513.3889731821114
Max note length: 10500


In [0]:
dictionary

{'the': 1,
 'and': 2,
 'to': 3,
 'of': 4,
 'was': 5,
 'with': 6,
 'a': 7,
 'on': 8,
 'in': 9,
 'for': 10,
 'no': 11,
 'mg': 12,
 'tablet': 13,
 'patient': 14,
 'is': 15,
 'he': 16,
 'at': 17,
 'blood': 18,
 'name': 19,
 'po': 20,
 'she': 21,
 'discharge': 22,
 'or': 23,
 's': 24,
 'as': 25,
 'day': 26,
 'daily': 27,
 'sig': 28,
 'one': 29,
 'his': 30,
 'left': 31,
 'last': 32,
 'history': 33,
 'her': 34,
 'were': 35,
 'right': 36,
 'by': 37,
 'had': 38,
 'be': 39,
 'this': 40,
 'admission': 41,
 'not': 42,
 'date': 43,
 'you': 44,
 'pt': 45,
 'pain': 46,
 'hospital': 47,
 'normal': 48,
 'your': 49,
 'an': 50,
 'that': 51,
 'first': 52,
 'from': 53,
 'p': 54,
 'there': 55,
 'has': 56,
 'are': 57,
 'ct': 58,
 'have': 59,
 'which': 60,
 'medications': 61,
 'but': 62,
 'up': 63,
 'chest': 64,
 'c': 65,
 'hospital1': 66,
 'd': 67,
 'hours': 68,
 'times': 69,
 'o': 70,
 'well': 71,
 'status': 72,
 'time': 73,
 'dr': 74,
 'also': 75,
 'home': 76,
 'please': 77,
 'given': 78,
 'stable': 79,
 '

In [0]:
#To save Dictionary
import pickle
with open('dictionary.pkl', 'wb') as f:
        pickle.dump(dictionary, f, pickle.HIGHEST_PROTOCOL)

In [0]:
#To load Dictionary
import pickle
with open(data_path+'dictionary.pkl', 'rb') as f:
        dictionary = pickle.load(f)

In [0]:
#---------------CONSTRUCTION OF EMBEDDING MATRIX USING BIOWORDVEC--------------------------#

In [0]:
EMBEDDING_DIM=200
EMBEDDING_MATRIX=np.zeros((len(dictionary)+1, 200))

In [0]:
!pip install fasttext

In [0]:
#Loading of BioWordVec Model
import fasttext
model = fasttext.load_model('/content/gdrive/My Drive/BE Project/Implementation/BioWordVec_PubMed_MIMICIII_d200.bin')

In [0]:
#Converting words to embeddings 
for word, i in dictionary.items():
        vector = model.get_word_vector(word)
        if vector is not None:
            EMBEDDING_MATRIX[i] = vector

In [0]:
np.savetxt('/content/gdrive/My Drive/BE Project-Base/embedding_matrix.csv', EMBEDDING_MATRIX, delimiter=',')

In [0]:
#------------------END OF EMBEDDING MATRIX SECTION--------------------------#

In [0]:
#--------------CONVERTING NOTES TO INTEGER VALUES ACCORDING TO DICTIONARY------------#

In [0]:
df_train_num= pd.read_csv(data_path+'disch_train_split.csv')

In [0]:
df_test_num= pd.read_csv(data_path+'disch_test_split.csv')

In [0]:
df_dev_num= pd.read_csv(data_path+'disch_dev_split.csv')

In [0]:
df_validation_num = pd.read_csv(data_path+'disch_validation_split.csv')

In [0]:
#Converting from Text to list of numbers accoring to dictionary
MAX_SEQ_LENGTH = 5000
def text_to_seq (vocab, data_sent, MAX_SEQ_LENGTH):

  index = data_sent.columns.get_loc("TEXT")
  for i, sent in enumerate(data_sent.TEXT):
    data_sent.iloc[i, index] = ''
    if i%100 == 0:
      print(i)
    for word in sent.split():
        data_sent.iloc[i, index] = data_sent.iloc[i, index] + str(vocab.get(word)) + ' '
    data_sent.at[i, 'TEXT'] = data_sent.iloc[i, index].split()
    if len(data_sent.at[i, 'TEXT']) > MAX_SEQ_LENGTH:
        data_sent.at[i, 'TEXT'] = data_sent.at[i, 'TEXT'][:MAX_SEQ_LENGTH]
    else:
        data_sent.at[i, 'TEXT'] += ['0'] * (MAX_SEQ_LENGTH - len(data_sent.at[i, 'TEXT']))
  return data_sent

In [0]:
# Convert the list elements in data.TEXT column from strings to integers
 def str_to_int (data_sent, MAX_SEQ_LENGTH):
        for x, sent in enumerate(data_sent.TEXT):
            if x%100 ==0:
                print(x)
            for y in range(0, MAX_SEQ_LENGTH):
                if data_sent.at[x, 'TEXT'][y]=='None':
                    data_sent.at[x, 'TEXT'][y]='0'
                data_sent.at[x, 'TEXT'][y] = int(data_sent.at[x, 'TEXT'][y])
        return data_sent

In [0]:
#For Training Split, we split df_train_num into parts and then run through text_to_seq and str_to_int
# Executing the code below for the entire of df_train_num will be lengthy
df_train_num= text_to_seq(dictionary, df_train_num, MAX_SEQ_LENGTH)
df_train_num = str_to_int(df_train_num, MAX_SEQ_LENGTH)

In [0]:
df_test_num= text_to_seq(dictionary, df_test_num, MAX_SEQ_LENGTH)
df_test_num = str_to_int(df_test_num, MAX_SEQ_LENGTH)

In [0]:
df_dev_num=text_to_seq(dictionary, df_test_num, MAX_SEQ_LENGTH)
df_dev_num=str_to_int(df_test_num, MAX_SEQ_LENGTH)

In [0]:
df_validation_num=text_to_seq(dictionary, df_validation_num, MAX_SEQ_LENGTH)
df_validation_num=str_to_int(df_validation_num, MAX_SEQ_LENGTH)

In [0]:
#Saving the Converted DATA
df_dev_num.to_csv(data_path+'test_split_num.csv', index=False)
df_dev_num.to_csv(data_path+'dev_split_num.csv', index=False)

In [0]:
df_validation_num.to_csv(data_path+'validation_split_num.csv', index=False)

In [0]:
df_validation_num= pd.read_csv(data_path+'validation_split_num.csv')
df_validation_num.head()

In [0]:
#------------------------END OF CONVERSION NOTES TO SEQUENCE----------------#

In [0]:
#---------------THESE PREPROCESSED FILES ARE THEN USED FOR MODEL TRAINING----------------#